<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
from __future__ import print_function

import sys
import requests
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random

In [2]:
# url = "https://www.gutenberg.org/files/100/100-0.txt"
# r = requests.get(url)
# r.encoding = r.apparent_encoding
# data = r.text

In [3]:
if 0 == 1:
    txt_data.splitlines()
    txt_data.rstrip('\n') # didnt work, but maybe it's better with them
    txt_data.split('\n\n\n')

In [4]:
# txt_data = open('sonnets.txt', 'r').read() # test external files

In [5]:
# chars = list(set(txt_data))
# num_chars = len(chars)
# txt_data_size = len(txt_data)
# print(f'Total characters: {txt_data_size}. Unique characters: {num_chars}.')

In [6]:
# char_to_int = dict((c, i) for i, c in enumerate(chars))
# int_to_char = dict((i, c) for i, c in enumerate(chars))
# integer_encoded = [char_to_int[i] for i in txt_data] # could be used for manual one-hot encode
# print(int_to_char)

I manually copied the block of 154 sonnets into notepad, then used find and replace to make the fancy apostrophes a normal character.

In [7]:
text = open('sonnets.txt', 'r').read()

In [8]:
for i in range(3):
    print(text.split('\n\n\n')[i])

                    1

From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own bright eyes,
Feed'st thy light's flame with self-substantial fuel,
Making a famine where abundance lies,
Thy self thy foe, to thy sweet self too cruel:
Thou that art now the world's fresh ornament,
And only herald to the gaudy spring,
Within thine own bud buriest thy content,
And, tender churl, mak'st waste in niggarding:
  Pity the world, or else this glutton be,
  To eat the world's due, by the grave and thee.
                    2

When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a tattered weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thr

In [9]:
chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

len(chars)

71

In [10]:
encoded = [char_int[c] for c in text]
len(encoded)

98328

In [11]:
maxlen = 512
step = 16

sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences:', len(sequences))

sequences: 6114


In [12]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

x.shape, y.shape

((6114, 512, 71), (6114, 71))

In [13]:
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

W0128 01:50:29.545747 14712 deprecation.py:506] From C:\Users\John\Anaconda3\envs\U4-S2-NN\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [15]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    if epoch % 32 == 0:
    
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)

        generated = ''

        sentence = text[start_index: start_index + maxlen]
        generated += sentence

        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(128):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature=1.0)
            next_char = int_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [16]:
model.fit(x, y,
          batch_size=1024,
          epochs=256,
          callbacks=[print_callback])

W0128 01:50:29.888653 14712 deprecation.py:323] From C:\Users\John\Anaconda3\envs\U4-S2-NN\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/256
5120/6114 [========================>.....] - ETA: 10s - loss: 4.2275
----- Generating text after Epoch: 0
----- Generating with seed: "ft you, mine eye is in my mind,
And that which governs me to go about,
Doth part his function, and is partly blind,
Seems seeing, but effectually is out:
For it no form delivers to the heart
Of bird, of flower, or shape which it doth latch,
Of his quick objects hath the mind no part,
Nor his own vision holds what it doth catch:
For if it see the rud'st or gentlest sight,
The most sweet favour or deformed'st creature,
The mountain, or the sea, the day, or night:
The crow, or dove, it shapes them to your feat"
ft you, mine eye is in my mind,
And that which governs me to go about,
Doth part his function, and is partly blind,
Seems seeing, but effectually is out:
For it no form delivers to the heart
Of bird, of flower, or shape which it doth latch,
Of his quick objects hath the mind no part,
Nor his own vision holds what it doth catch:
For if it

In [17]:
start_index = random.randint(0, len(text) - maxlen - 1)

generated = ''

sentence = text[start_index: start_index + maxlen]
generated += sentence

print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

for i in range(2048):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_int[char]] = 1

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, temperature=1.0)
    next_char = int_char[next_index]

    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()
print()

----- Generating with seed: "y precious minutes waste,
These vacant leaves thy mind's imprint will bear,
And of this book, this learning mayst thou taste.
The wrinkles which thy glass will truly show,
Of mouthed graves will give thee memory,
Thou by thy dial's shady stealth mayst know,
Time's thievish progress to eternity.
Look what thy memory cannot contain,
Commit to these waste blanks, and thou shalt find
Those children nursed, delivered from thy brain,
To take a new acquaintance of thy mind.
  These offices, so oft as thou wilt loo"
y precious minutes waste,
These vacant leaves thy mind's imprint will bear,
And of this book, this learning mayst thou taste.
The wrinkles which thy glass will truly show,
Of mouthed graves will give thee memory,
Thou by thy dial's shady stealth mayst know,
Time's thievish progress to eternity.
Look what thy memory cannot contain,
Commit to these waste blanks, and thou shalt find
Those children nursed, delivered from thy brain,
To take a new acquaintanc

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN